In [1]:
import keras,os
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D , Flatten, Dropout
from keras.preprocessing.image import ImageDataGenerator
import numpy as np

In [2]:
import cv2
import numpy as np
import os
import pickle
from sklearn.model_selection import train_test_split

def get_data(data_loaded, test_split_ratio):
    
    x = []
    y = []
    
    for im, lb in data_loaded:
        x.append(im)
        y.append(lb)
    
    x = np.array(x, dtype = np.float32)
    x = x/255.0
    y = np.array(y)
    
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size= test_split_ratio, random_state=42)
    return [X_train, X_test, y_train, y_test]


def load_pickle_file(path, test_split_ratio = 0.33):
    pick = open(path, 'rb')
    data_laoded = pickle.load(pick)
    pick.close()
    return get_data(data_loaded, test_split_ratio)


def make_data(data_dir, categories, im_size = (224, 224) , save_data=False, test_split_ratio=0.33):
    
    data = []
    # making data to store [im, label] for each images in every directory - then can shuffle data keeping every [im, label] pair intact
    for category in categories:
        path = os.path.join(data_dir, category).replace("\\","/")
        label = categories.index(category)
        
        for im_name in os.listdir(path):
            im_path = os.path.join(path, im_name).replace("\\","/")
            
            try: 
                im = cv2.imread(im_path)
                im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
                im = cv2.resize(im, im_size)
                im = np.array(im, dtype = np.float32)
                data.append([im, label])
                
                
            except Exception as e:
                print("error in loading images")
                pass
        
    np.random.shuffle(data)
    print("Total data length: " + str(len(data)))
    if(save_data):
        pick = open('data.pickle', 'wb')
        pickle.dump(data, pick)
        pick.close()
        
    return get_data(data, test_split_ratio)
    

In [3]:
data_dir = './datasets/flowers_small'
categories = ['daisy', 'dandelion', 'rose', 'sunflower','tulip']
im_size = (224, 224)

X_train, X_test, y_train, y_test = make_data(data_dir, categories, im_size, False, 0.33)


Total data length: 500


In [4]:
dataAugmentaion = ImageDataGenerator(rotation_range = 30, zoom_range = 0.20, 
fill_mode = "nearest", shear_range = 0.20, horizontal_flip = True, 
width_shift_range = 0.1, height_shift_range = 0.1)

In [5]:
model1 = Sequential([
    keras.Input(shape=(224, 224, 3 )),
    
    Conv2D(filters = 64, kernel_size=(3,3), strides=1, padding="same", activation="relu"),
    Conv2D(filters = 64, kernel_size=(3,3), strides=1, padding="same", activation="relu"),
    MaxPooling2D(pool_size=(3,3), strides=2),
    
    Conv2D(filters = 128, kernel_size=(3,3), strides=1, padding="same", activation="relu"),
    Conv2D(filters = 128, kernel_size=(3,3), strides=1, padding="same", activation="relu"),
    MaxPooling2D(pool_size=(3,3), strides=2),
    
    Conv2D(filters = 256, kernel_size=(3,3), strides=1, padding="same", activation="relu"),
    Conv2D(filters = 256, kernel_size=(3,3), strides=1, padding="same", activation="relu"),
    Conv2D(filters = 256, kernel_size=(3,3), strides=1, padding="same", activation="relu"),
    MaxPooling2D(pool_size=(3,3), strides=2),
    
    Conv2D(filters = 512, kernel_size=(3,3), strides=1, padding="same", activation="relu"),
    Conv2D(filters = 512, kernel_size=(3,3), strides=1, padding="same", activation="relu"),
    Conv2D(filters = 512, kernel_size=(3,3), strides=1, padding="same", activation="relu"),
    MaxPooling2D(pool_size=(3,3), strides=2),
    
    Conv2D(filters = 512, kernel_size=(3,3), strides=1, padding="same", activation="relu"),
    Conv2D(filters = 512, kernel_size=(3,3), strides=1, padding="same", activation="relu"),
    Conv2D(filters = 512, kernel_size=(3,3), strides=1, padding="same", activation="relu"),
    MaxPooling2D(pool_size=(3,3), strides=2),
    
    Flatten(),
    
    Dense(4096, activation="relu"),
    Dropout(0.5),
    Dense(4096, activation="relu"),
    Dropout(0.5),
    Dense(5, activation="softmax") 
])

In [6]:
from keras.optimizers import Adam
opt = Adam(learning_rate = 0.01)
model1.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [7]:
model1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 224, 224, 64)      1792      
                                                                 
 conv2d_1 (Conv2D)           (None, 224, 224, 64)      36928     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 111, 111, 64)     0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 111, 111, 128)     73856     
                                                                 
 conv2d_3 (Conv2D)           (None, 111, 111, 128)     147584    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 55, 55, 128)      0         
 2D)                                                    

In [63]:
def one_hot_encode(y):
    b = np.zeros((y.size, y.max()+1))
    b[np.arange(y.size), y] = 1
    print(b.shape)
    return b

In [64]:
y_train_1 = one_hot_encode(y_train)
y_test_1 = one_hot_encode(y_test)

(335, 5)
(165, 5)


In [11]:
trdataAugmentaion = ImageDataGenerator(rotation_range = 30, zoom_range = 0.20, 
fill_mode = "nearest", shear_range = 0.20, horizontal_flip = True, 
width_shift_range = 0.1, height_shift_range = 0.1)

trdata = trdataAugmentaion.flow(X_train, y_train)

tsdataAugmentaion = ImageDataGenerator(rotation_range = 30, zoom_range = 0.20, 
fill_mode = "nearest", shear_range = 0.20, horizontal_flip = True, 
width_shift_range = 0.1, height_shift_range = 0.1)

tsdata = tsdataAugmentaion.flow(X_train, y_train)

In [15]:
import tensorflow as tf
tf.config.run_functions_eagerly(True)

In [16]:
model1.fit(trdata, epochs = 10, validation_data=tsdata )

Epoch 1/10
 1/11 [=>............................] - ETA: 2:50 - loss: 14691.1221 - accuracy: 0.1875

KeyboardInterrupt: 